In [2]:
import numpy as np
import tensorflow as tf
import re
import time

DATA PreProcessing

In [3]:
#Importing dataset
lines=open('movie_lines.txt',encoding='utf-8',errors='ignore').read().split('\n')
conversations=open('movie_conversations.txt',encoding='utf-8',errors='ignore').read().split('\n')

In [4]:
print(len(lines))

304714


In [5]:
#creating a dictionary with maps each line with its id
id2line={}
for line in lines:
    temp_line=line.split(' +++$+++ ')
    if(len(temp_line)==5):
        id2line[temp_line[0]]=temp_line[4]

In [6]:
print(len(id2line.keys()))

304713


In [7]:
#creating list of all conversations
conversations_ids=[]
for conv in conversations[:-1]:
    conv_temp=conv.split(' +++$+++ ')[-1][1:-1].replace("'","").replace(" ","")
    conversations_ids.append(conv_temp.split(','))
    

In [8]:
#Getting separately the questions and answers
questions=[]
answers=[]
for conversation in conversations_ids:
    for i in range(len(conversation)-1):
        questions.append(id2line[conversation[i]])
        answers.append(id2line[conversation[i+1]])

In [9]:
print(questions[0])
print(answers[0])

Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.
Well, I thought we'd start with pronunciation, if that's okay with you.


In [10]:
#CLeaning

In [11]:

def clean_text(text):
    text=text.lower()
    text=re.sub(r"i'm","i am",text)
    text=re.sub(r"he's","he is",text)
    text=re.sub(r"she's","she is",text)
    text=re.sub(r"that's","that is",text)
    text=re.sub(r"what's","what is",text)
    text=re.sub(r"where's","where is",text)
    text=re.sub(r"\'ll","will",text)
    text=re.sub(r"\'ve","have",text)
    text=re.sub(r"\'re","are",text)
    text=re.sub(r"\'d","would",text)
    text=re.sub(r"won't","will not",text)
    text=re.sub(r"don't","do not",text)
    text=re.sub(r"can't","cannot",text)
    text=re.sub(r"it's","it is",text)
    text=re.sub(r"[-()\#/@;<>{}+=|.?,]","",text)
    return text
    

In [12]:
clean_questions=[]
for question in questions:
    clean_questions.append(clean_text(question))

clean_answers=[]
for answer in answers:
    clean_answers.append(clean_text(answer))

In [13]:
print(clean_answers[700])

what about back home


In [14]:
#create a dictionary that maps each words to its number of occurences
word2count={}
for question in clean_questions:
    for word in question.split():
        if(word not in word2count):
            word2count[word]=1
        else:
            word2count[word]+=1

for answer in clean_answers:
    for word in answer.split():
        if(word not in word2count):
            word2count[word]=1
        else:
            word2count[word]+=1
    

In [15]:
word2count['i']

182808

In [16]:
#creating 2 dictionaries which map the questions words and the answers word to a integer and filter out words which are boyond a threshold
threshold=20
questionswords2int={}
word_number=0
for word,count in word2count.items():
    if(count>threshold):
        questionswords2int[word]=word_number
        word_number+=1


answerswords2int={}
word_number=0
for word,count in word2count.items():
    if(count>threshold):
        answerswords2int[word]=word_number
        word_number+=1

In [17]:
#Adding more tokens

tokens=['<PAD>','<EOS>','<OUT>','<SOS>']
for token in tokens:
    questionswords2int[token]=len(questionswords2int)+1

for token in tokens:
    answerswords2int[token]=len(answerswords2int)+1

In [18]:
#inverse dictionary
answersint2word={w_i:w for w,w_i in answerswords2int.items()}
#questionsint2word={w_i:for w,w_i in questionswords2int.items()}

In [19]:
#add EOS to end of every answer
for i in range(len(clean_answers)):
    clean_answers[i]+=' <EOS>'

In [20]:
#translated all the questions and answers into integers
questions2int=[]
for question in clean_questions:
    ints=[]
    for word in question.split():
        if(word not in questionswords2int):
            ints.append(questionswords2int['<OUT>'])
        else:
            ints.append(questionswords2int[word])
    questions2int.append(ints)
    
answers2int=[]
for answer in clean_answers:
    ints=[]
    for word in question.split():
        if(word not in answerswords2int):
            ints.append(answerswords2int['<OUT>'])
        else:
            ints.append(answerswords2int[word])
    answers2int.append(ints)

In [21]:
#Sorting questions and answers by length of questions
sorted_clean_questions=[]
sorted_clean_answers=[]

for length in range(1,25+1):
    for i in enumerate(questions2int):
        if(len(i[1])==length):
            sorted_clean_questions.append(questions2int[i[0]])
            sorted_clean_answers.append(answers2int[i[0]])
           

In [22]:
sorted_clean_answers[0]

[5,
 18,
 1681,
 27,
 27,
 94,
 28,
 111,
 1643,
 285,
 67,
 5241,
 93,
 38,
 1443,
 1619,
 37,
 45,
 38,
 6005,
 595]

In [39]:
sorted_clean_questions[0]

[48]

In [83]:
import tensorflow as tf
